In [1]:
import os
import numpy as np
import pandas as pd
import pyranges as pr
from utils.ref import get_tss_pyranges

In [2]:
pth = os.path.join('..', '..', '..', 'data')
pth_in = os.path.join(pth, 'unzip')
pth_eom = os.path.join(pth_in, 'GSE272849')
pth_bra = os.path.join(pth_in, 'GSE128466')
pth_ref = os.path.join(pth, 'ref')
pth_mod = os.path.join(pth, 'features', 'biomart', 'modules')
pth_out = os.path.join(pth, 'DELAY', 'GSE162534')

In [3]:
g = pd.read_csv(os.path.join(pth_mod, 'union.csv')).mmusculus
tss = get_tss_pyranges(os.path.join(pth_ref, 'mm10.refGene.gtf.gz'))

/home/caleb/MesenCoder/DELAY/preprocessing/ground-truth/utils/ref.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_upstream_tss))


In [4]:
g1, g2 = [], []

# Eomes
peaks = pr.read_bed(os.path.join(pth_eom, 'Eomes.txt'))
tss_peaks = peaks.nearest(tss, apply_strand_suffix = True)
g_peaks = tss_peaks.df.gene_id.unique()
tgt = g_peaks[np.isin(g_peaks, g)].tolist()
g1.extend(['Eomes'] * len(tgt)); g2.extend(tgt)

# Brachyury
fn = os.path.join(pth_bra, 'T.txt')
g_peaks = pd.read_csv(fn, header = None)[0].unique()
tgt = g_peaks[np.isin(g_peaks, g)].tolist()
g1.extend(['T'] * len(tgt)); g2.extend(tgt)

# write output
refNet = (pd.DataFrame({'Gene1' : g1, 'Gene2' : g2})
          .drop_duplicates()
          .sort_values(['Gene1', 'Gene2']))
fn = os.path.join(pth_out, 'refNetwork.csv')
refNet.to_csv(fn, index = False); refNet

,Gene1,Gene2
45,Eomes,A2m
89,Eomes,Actn1
29,Eomes,Agtrap
70,Eomes,Alx1
15,Eomes,Alx4
...,...,...
210,T,Tnfrsf19
172,T,Trh
215,T,Ubl4b
181,T,Wnt2


In [5]:
n_edges = refNet.groupby('Gene1').size()
density = (n_edges / g.shape[0])
density.sort_values(ascending = False)

Gene1
Eomes    0.274633
T        0.194969
dtype: float64